           INSTITUTO TECNOLÓGICO DE ESTUDIOS SUPERIORES DEL OCCIDENTE

              DEPARTAMENTO DE PROCESOS TECNOLÓGICOS E INDUSTRIALES

                     PROYECTO DE APLICACIÓN PROFESIONAL (PAP)

                    SISTEMA DE PREDICCIÓN DE CALIDAD DE AIRE

                       
                       
                    "TRATAMIENTO DE BASE DE DATOS 2015-2017"



                                   PRESENTAN:

                         I.Q. VICTOR DAVID ANDARCIA LEZAMA

                            I.Q. JOSE LUIS IRIGOYEN 


# INTRODUCCIÓN

A continuación se muestra el procedimiento paso a paso para la ejecución del código "final.py" elaborado por los estudiantes de Ingeniería Química. Éste código tiene como finalidad producir 15 documentos en formato .csv que contengan en las columnas todas las estaciones de monitoreo y las filas representen los valores para una fecha-hora determinada. Tomando como documentos iniciales los otorgados por la SEMADET, en estos se separan las estaciones de monitoreo por pestaña en el documento y dentro de estas pestañas se encuentran las mediciones de todos los contaminantes.

Como pasos previos es necesario asegurarse que:

1) Los documentos de excel se encuentren en el escritorio de trabajo del código (Si se desconoce como realizar este proceso, revisar módulos .glob y .cwd de la biblioteca os).

2) Los nombres de los contaminantes y los nombres de las estaciones sean completamente iguales a los presentes en el código para una ejecución correcta. 



# DECLARACIÓN DE LIBRERÍAS
Para comprender el funcionamiento del código, es necesario conocer lo básico de las siguientes librerías, ya que serán la base del código presentado.


In [1]:
import pandas as pd
from datetime import datetime, date, time, timedelta
import numpy as np


# DECLARACIÓN DE FUNCIONES

A continuación se mostrarán las funciones utilizadas a lo largo del código, estas cuentan con una descripción de su uso, sin embargo, se profundizará más en alguna de ellas para contextualizar al lector:


In [3]:
def Cambio_Hora(Hour):
    '''Función hecha para cambiar el formato de hora de los datos a un formato legible para biblioteca datetime.
    Se utiliza el formato Try/Except para evitar errores debido a que en la conversión de excel a pandas ya se haya considerado como hora.
    Entrada:
        date: Fecha en formato H/M/S
    Salida:
        datitme_obj.time(): objeto tipo time con todas las propiedades de biblioteca
        '''
    try:
        return Hour.time()
    except AttributeError:
        return Hour
    
      
def Cambio_Formato(df):
    '''Función hecha para cambiar el formato de fecha hora de los datos a un formato legible para biblioteca datetime.
    Entrada:
        df: Dataframe con formato de fecha y hora de excel.
    Salida:
        df: Mismo Dataframe pero con columna extra de fecha - hora en datatime object.
        '''
    #Creación de listas en donde guardar datos de conversión de formato
    p = list()
    c = list()
    for i in range(len(df)):
        p.append(df["Fecha"][i].date()) #Se obtiene la fecha en formato datetime para cada uno de los datos del dataframe (df).
        c.append(Cambio_Hora(df["Hora"][i])) #Se llama a la función Cambio_Hora para obtener la hora en formato datetime.
    
    #Eliminación de columnas de fecha y hora de excel
    df = df.drop(columns = "Fecha")
    df = df.drop(columns = "Hora")
    
    #Anexo de Fecha y Hora en formato datetime
    df["Fecha"] = p
    df["Hora"] = c
    
    #Union de fecha y hora para ahorro de espacio
    b = list()
    for i in range(len(df)):
        b.append(datetime.combine(df["Fecha"][i],df["Hora"][i]))
        
    df["Fecha - Hora"] = b
    
    return df


def FechasCompletas(df):
    
    '''Función que elabora un DataFrame con valores de 1 en todas las estaciones para comparar contra los datos reales.
    Entrada:
        df: Dataframe con formato de fecha y hora en datetime.
    Salida:
        DFCompleto: DataFrame con las dimensiones ideales de mediciones totales en blanco. 
        '''
    d1 = df["Fecha - Hora"].iloc[0] #Posición incial 
    d2 = df["Fecha - Hora"].iloc[-1] #Posición final
    
    dias = pd.date_range(d1,d2, freq="H") #crea una serie con todos los días presentes entre d1 y d2 separados por una hora.
    
    DFCompleto = pd.DataFrame({"Fecha - Hora":dias})
    
    Estaciones = ["Atemajac","Aguilas","Centro","Las Pintas","Loma dorada","Miravalle","Oblatos","Santa fe", "Tlaquepaque","Vallarta"]
    
    #llenado de valores con 1 en todas las celdas
    for i in Estaciones:
        j = np.empty(len(DFCompleto))
        j[:] = 9999
        DFCompleto[i] = j
        
    return DFCompleto


def DatosTotales(df,FC):  
    '''Función que elabora un DataFrame con todos los valores presentes en un año en la estación, 
    el resultado incluye hasta los días o horas en las que no se registraron datos.
    
    Entrada:
        df: Dataframe con formato de fecha y hora en datetime.
        FC: Dataframe con todas las fechas posibles en el año. 
    Salida:
        final: DataFrame con las dimensiones ideales de mediciones totales. 
        '''
    #Se fija el índice con el cual se unirán los datos
    df.set_index(pd.to_datetime(df['Fecha - Hora']), inplace=True)
    FC.set_index(pd.to_datetime(FC['Fecha - Hora']), inplace=True)
    
    merge = df.join(FC, how= "outer",lsuffix='', rsuffix='_other')  #Se escoge union "outer" para que aparezcan todas las posibles combinaciones.
    
    #Creación de DataFrame "Final" en donde se guardarán todos los datos unidos
    final = merge[["Fecha","Hora",'O3',"NO","NO2","NOX","SO2","CO","PM10","PM 2.5",
                   "Temp Ext","RH","WS","WD","Precipitacion","Rad Solar","Presion"]]
    
    final.drop_duplicates(keep=False,inplace=True) #Eliminación de todos los duplicados para evitar errores de concatenación.
    
    return final


def Proceso(df):
    '''Función que concecta las funciones Cambio_Formato y DatosTotales para producir el DataFrame final con todos los datos presentes en un año.
    
    Entrada:
        df: Dataframe con formato de fecha y hora en formato excel. 
    Salida:
        final: DataFrame con las dimensiones ideales de mediciones totales. 
        '''
    Zona = Cambio_Formato(df)
    return DatosTotales(Zona, FechasCompletas(Zona))


def AcomodoReal(df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,P):
    """Función encargada de acoplar todos los datos de un contaminante específico en un solo DataFrame, las columnas representarán las estaciones.
        Entrada:
        df(1-10): DataFrames de zonas de medición.
        P: parámetro o contaminante a evaluar
    Salida:
        d: DataFrame de contaminante P ordenado por estaciones.
        """
    Estaciones = ["Atemajac","Aguilas","Centro","Las_Pintas","Loma_Dorada","Miravalle","Oblatos","Santa_Fe", "Tlaquepaque","Vallarta"]
    
    d = df1[P].to_frame(name=Estaciones[0]) #Se establece la primera estación en el dataframe
    d[Estaciones[1]]= df2[P].to_frame()#El resto de las estaciones solo se agrega al dataframe "d"
    d[Estaciones[2]]= df3[P].to_frame()
    d[Estaciones[3]]= df4[P].to_frame()
    d[Estaciones[4]]= df5[P].to_frame()
    d[Estaciones[5]]= df6[P].to_frame()
    d[Estaciones[6]]= df7[P].to_frame()
    d[Estaciones[7]]= df8[P].to_frame()
    d[Estaciones[8]]= df9[P].to_frame()
    d[Estaciones[9]]= df10[P].to_frame()

        
    return d


def VentanaMovil(P,limsup,lim):
    """Función encargada de eliminar datos espurios de acuerdo a los límites de medición y de realizar un promedio de ventana móvil de seis horas 
    para rellenar datos incompletos.
    Entrada:
        P: DataFrame con todos los datos de contaminantes por zona.
        limsup: Límite de medición del instrumento.
        lim: Límite de salud aceptado por la NOM. 
    Salida:
        Final: DataFrame de contaminante P con nuevos valores promediados.
    """
    P.reset_index(inplace=True)
    P.replace(0, np.nan, inplace=True) 
    P.replace("ND",np.nan, inplace=True)
    Estaciones = ["Atemajac","Aguilas","Centro","Las_Pintas","Loma_Dorada","Miravalle","Oblatos","Santa_Fe", "Tlaquepaque","Vallarta"]
    
    final = pd.DataFrame()
    final["Fecha - Hora"]= P["Fecha - Hora"]
    
    for Estacion in Estaciones:
        df = P[Estacion]
        
        #Eliminación de valores espurios o mal registrados. Se propone utilizar Try/Except par evitar errores con primeros y últimos datos:
        for i in range(len(df)):
        
        #Eliminación de datos mayores a los posibles medidos por máquina (var limsup)
            try:
                if df[i] > limsup: 
                    df[i] = np.nan
                    continue
            #No se pueden comparar datos NaN, por lo que se propone el siguiente código.
            
            #Eliminación de datos por diferencias mayores al lim de la norma (var lim)
                if df[i] > df[i+1]:
                    if (df[i]-df[i+1]) > lim:
                        df[i] = np.nan
                        continue
                        
        #Si el valor siguiente es mayor que el anterior por una diferencia de lim.
                if df[i] < df[i+1]:
                    if (df[i+1]-df[i]) > lim:
                        df[i+1] = np.nan
                        continue
                        
            except KeyError:
                next
        #Promedio de ventana móvil de 6 horas. Se propone utilizar Try/Except par evitar errores con primeros y últimos datos:   
        contador = 0
        for i in range(len(df)):
            try:
                if np.isnan(df[i-1]):
                #Si el valor de una celda antes de la evaluada es NaN, significa que el contador ya llegó a su máximo de 6. 
                #Por lo que a partir de éste dato no se puede estimar.
                    contador = 0
                    continue
                
                if np.isnan(df[i]) == True & (contador < 7) == True:
                    #Para los valores que si cumplen con las instrucciones anteriores se les hará la siguiente prueba, si todos son diferentes a NaN se puede hacer el promedio:
                    if not np.isnan(df[i-6]) & np.isnan(df[i-5]) & np.isnan(df[i-4]) & np.isnan(df[i-3]) & np.isnan(df[i-2]) & np.isnan(df[i-1]):      
                        df[i] = df[i-6:i].mean()
                        contador += 1
                        continue
                 #Reinicio de contador    
                contador = 0
                
            except KeyError:
                next
        #Se anexa al DataFrame final los valores encontados        
        final[Estacion] = df[:]
        
    return final



def Normalizacion(DF,P):
    """Función encargada de normalizar los datos de contaminantes, es necesario conocer los valores límites de la NOM para cada contaminante así como los mínimos detectables por el equipo de medición.
    
    Entrada:
        DF: DataFrame con todos los datos de contaminantes por zona.
        P: Parámetro de medición (va de 1 a 15 de acuerdo al parámetro enlistado a continuación):
        Parametros = ['O3',"NO","NO2","NOX","SO2","CO","PM10","PM 2.5",
              "Temp Ext","RH","WS","WD","Precipitacion","Rad Solar","Presion"]
    Salida:
        Final: DataFrame de parámetro P con nuevos valores normalizados.
    """

    Lim_Norma = [0.095, 0.395,0.210,0.395,0.288, 11, 75, 45, 45, 100,30,360, 1000,1000,1000]
    Lim_Min = [0.5/1000, 4/1000, 4/1000, 4/1000, 0.5/1000, 0.05, 4, 4,0, 0,0,0,0,0,0] 
    
    Estaciones = ["Atemajac","Aguilas","Centro","Las_Pintas","Loma_Dorada","Miravalle","Oblatos","Santa_Fe", "Tlaquepaque","Vallarta"]
    
    final = pd.DataFrame()
    final["Fecha - Hora"]= DF["Fecha - Hora"]

    for Estacion in Estaciones:
        temp = pd.Series(float(-10), index=range(len(final)))
        df = DF[Estacion]
        
        for i in range(len(df)):
            if np.isnan(df[i]) == True:
                continue
            else:
                #Para los valores que si cumplen con las instrucciones anteriores se les aplicará la fórmula de normalización:
                temp[i] = float(1 + (df[i]-Lim_Norma[P])/(Lim_Norma[P]-Lim_Min[P]))
            #Se anexa al DataFrame final los valores encontados        
        final[Estacion] = temp[:]
            
    return final



# PROCESAMIENTO DE DATOS

Con las funciones ya explicadas, proseguimos a la ejecución del código. La primera etapa consiste en extraer del excel los datos de contaminantes con ayuda del módulo "pd.read_excel()". Se crearán 10 dataframes, cada uno conteniendo los valores por estación de monitoreo, estos valores se convertirán a formatos legibles por la biblioteca datetime y por pandas.

Ya que contamos con los valores de contaminantes por estación en el formato adecuado proseguimos a juntar cada contaminante en un dataframe, por lo que ahora existirán 15 dataframes con las concentraciones del contaminante objetivo en todas las estaciones. 

Este proceso se aplicará para los años 2015, 2016 y 2017. A continuación se muestran los códigos:



In [1]:
#%%   2015
a = pd.read_excel("Datos_2015.xlsx", sheet_name=None)
#Todas las pestañas se converten en diccionarios, por lo que es necesario llamarlas

## Todo lo siguiente puede ser una funci'on 
Estaciones = ["Atemajac","Águilas","Centro","Las Pintas","Loma Dorada","Miravalle","Oblatos","Santa Fe", "Tlaquepaque","Vallarta"]

Atemajac = a[Estaciones[0]]
Atemajac_2015 = Proceso(Atemajac)

Aguilas = a[Estaciones[1]]
Aguilas_2015 = Proceso(Aguilas)

Centro = a[Estaciones[2]]
Centro_2015 = Proceso(Centro)

Las_Pintas = a[Estaciones[3]]
Las_Pintas_2015 = Proceso(Las_Pintas)

Loma_dorada = a[Estaciones[4]]
Loma_dorada_2015 = Proceso(Loma_dorada)

Miravalle = a[Estaciones[5]]
Miravalle_2015 = Proceso(Miravalle)

Oblatos = a[Estaciones[6]]
Oblatos_2015 = Proceso(Oblatos)

Santa_Fe = a[Estaciones[7]]
Santa_Fe_2015 = Proceso(Santa_Fe)

Tlaquepaque = a[Estaciones[8]]
Tlaquepaque_2015 = Proceso(Tlaquepaque)

Vallarta = a[Estaciones[9]]
Vallarta_2015 = Proceso(Vallarta)

del a
del Atemajac
del Aguilas
del Centro
del Las_Pintas
del Loma_dorada
del Miravalle
del Oblatos
del Santa_Fe
del Tlaquepaque
del Vallarta

#todo esto puede ser una funcion de una clase 
Parametros = ['O3',"NO","NO2","NOX","SO2","CO","PM10","PM 2.5",
              "Temp Ext","RH","WS","WD","Precipitacion","Rad Solar","Presion"]

O3_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[0])

NO_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[1])

NO2_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[2])

NOX_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[3])

SO2_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[4])

CO_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[5])

PM10_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[6])

PM2_5_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[7])

TEMP_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[8])

RH_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[9])

WS_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[10])

WD_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[11])

Pr_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[12])

R_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[13])

Pa_2015 = AcomodoReal(Atemajac_2015,Aguilas_2015,Centro_2015,
                      Las_Pintas_2015,Loma_dorada_2015,
                      Miravalle_2015,Oblatos_2015,Santa_Fe_2015,
                      Tlaquepaque_2015,Vallarta_2015,Parametros[14])

del Atemajac_2015
del Aguilas_2015
del Centro_2015
del Las_Pintas_2015
del Loma_dorada_2015
del Miravalle_2015
del Oblatos_2015
del Santa_Fe_2015
del Tlaquepaque_2015
del Vallarta_2015
    



NameError: name 'pd' is not defined

In [ ]:
#%%   2016
a = pd.read_excel("Datos_2016.xlsx", sheet_name=None)
#Todas las pestañas se converten en diccionarios, por lo que es necesario llamarlas

Estaciones = ["Atemajac","Águilas","Centro","Las Pintas","Loma Dorada","Miravalle","Oblatos","Santa Fe", "Tlaquepaque","Vallarta"]

#Prueba, primera estación
Atemajac = a[Estaciones[0]]
Atemajac_2016 = Proceso(Atemajac)

Aguilas = a[Estaciones[1]]
Aguilas_2016 = Proceso(Aguilas)

Centro = a[Estaciones[2]]
Centro_2016 = Proceso(Centro)

Las_Pintas = a[Estaciones[3]]
Las_Pintas_2016 = Proceso(Las_Pintas)

Loma_dorada = a[Estaciones[4]]
Loma_dorada_2016 = Proceso(Loma_dorada)

Miravalle = a[Estaciones[5]]
Miravalle_2016 = Proceso(Miravalle)

Oblatos = a[Estaciones[6]]
Oblatos_2016 = Proceso(Oblatos)

Santa_Fe = a[Estaciones[7]]
Santa_Fe_2016 = Proceso(Santa_Fe)

Tlaquepaque = a[Estaciones[8]]
Tlaquepaque_2016 = Proceso(Tlaquepaque)

Vallarta = a[Estaciones[9]]
Vallarta_2016 = Proceso(Vallarta)

del a
del Atemajac
del Aguilas
del Centro
del Las_Pintas
del Loma_dorada
del Miravalle
del Oblatos
del Santa_Fe
del Tlaquepaque
del Vallarta

Parametros = ['O3',"NO","NO2","NOX","SO2","CO","PM10","PM 2.5",
              "Temp Ext","RH","WS","WD","Precipitacion","Rad Solar","Presion"]

O3_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[0])

NO_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[1])

NO2_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[2])

NOX_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[3])

SO2_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[4])

CO_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[5])

PM10_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[6])

PM2_5_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[7])

TEMP_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[8])

RH_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[9])

WS_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[10])

WD_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[11])

Pr_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[12])

R_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[13])

Pa_2016 = AcomodoReal(Atemajac_2016,Aguilas_2016,Centro_2016,
                      Las_Pintas_2016,Loma_dorada_2016,
                      Miravalle_2016,Oblatos_2016,Santa_Fe_2016,
                      Tlaquepaque_2016,Vallarta_2016,Parametros[14])

del Atemajac_2016
del Aguilas_2016
del Centro_2016
del Las_Pintas_2016
del Loma_dorada_2016
del Miravalle_2016
del Oblatos_2016
del Santa_Fe_2016
del Tlaquepaque_2016
del Vallarta_2016

In [ ]:
#%%   2017
a = pd.read_excel("Datos_2017.xlsx", sheet_name=None)
#Todas las pestañas se converten en diccionarios, por lo que es necesario llamarlas

Estaciones = ["Atemajac","Águilas","Centro","Las Pintas","Loma dorada","Miravalle","Oblatos","Santa Fe", "Tlaquepaque","Vallarta"]

#Prueba, primera estación
Atemajac = a[Estaciones[0]]
Atemajac_2017 = Proceso(Atemajac)

Aguilas = a[Estaciones[1]]
Aguilas_2017 = Proceso(Aguilas)

Centro = a[Estaciones[2]]
Centro_2017 = Proceso(Centro)

Las_Pintas = a[Estaciones[3]]
Las_Pintas_2017 = Proceso(Las_Pintas)

Loma_dorada = a[Estaciones[4]]
Loma_dorada_2017 = Proceso(Loma_dorada)

Miravalle = a[Estaciones[5]]
Miravalle_2017 = Proceso(Miravalle)

Oblatos = a[Estaciones[6]]
Oblatos_2017 = Proceso(Oblatos)

Santa_Fe = a[Estaciones[7]]
Santa_Fe_2017 = Proceso(Santa_Fe)

Tlaquepaque = a[Estaciones[8]]
Tlaquepaque_2017 = Proceso(Tlaquepaque)

Vallarta = a[Estaciones[9]]
Vallarta_2017 = Proceso(Vallarta)

del a
del Atemajac
del Aguilas
del Centro
del Las_Pintas
del Loma_dorada
del Miravalle
del Oblatos
del Santa_Fe
del Tlaquepaque
del Vallarta

Parametros = ['O3',"NO","NO2","NOX","SO2","CO","PM10","PM 2.5",
              "Temp Ext","RH","WS","WD","Precipitacion","Rad Solar","Presion"]

O3_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[0])

NO_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[1])

NO2_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[2])

NOX_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[3])

SO2_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[4])

CO_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[5])

PM10_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[6])

PM2_5_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[7])

TEMP_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[8])

RH_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[9])

WS_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[10])

WD_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[11])

Pr_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[12])

R_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[13])

Pa_2017 = AcomodoReal(Atemajac_2017,Aguilas_2017,Centro_2017,
                      Las_Pintas_2017,Loma_dorada_2017,
                      Miravalle_2017,Oblatos_2017,Santa_Fe_2017,
                      Tlaquepaque_2017,Vallarta_2017,Parametros[14])

del Atemajac_2017
del Aguilas_2017
del Centro_2017
del Las_Pintas_2017
del Loma_dorada_2017
del Miravalle_2017
del Oblatos_2017
del Santa_Fe_2017
del Tlaquepaque_2017
del Vallarta_2017

# CONCATENACIÓN Y NORMALIZACIÓN DE DATOS

Ya que contamos con los dataframes de contaminantes, el siguiente paso es unir todos para tener un solo dataframe para los 3 años por contaminante.

Despues de realizar la acción anterior, se elabora el promedio de ventana movil para rellenar datos faltantes y eliminar datos espurios.

Por último, para normalizar los datos se llama a la función "Normalizacion" para que los datos queden en un rango similar. 


In [ ]:
#%% DATOS TOTALES

#Es necesario correr las celdas anteriores

O3_Total = pd.concat([O3_2015,O3_2016,O3_2017])
NO_Total = pd.concat([NO_2015, NO_2016,NO_2017])
NO2_Total = pd.concat([NO2_2015, NO2_2016,NO2_2017])
NOX_Total = pd.concat([NOX_2015, NOX_2016, NOX_2017])
SO2_Total = pd.concat([SO2_2015, SO2_2016,SO2_2017])
CO_Total = pd.concat([CO_2015, CO_2016,CO_2017])
PM10_Total = pd.concat([PM10_2015, PM10_2016,PM10_2017])
PM2_5_Total = pd.concat([PM2_5_2015, PM2_5_2016,PM2_5_2017])
TEMP_Total = pd.concat([TEMP_2015, TEMP_2016,TEMP_2017])
RH_Total = pd.concat([RH_2015, RH_2016,RH_2017])
WS_Total = pd.concat([WS_2015, WS_2016,WS_2017])
WD_Total = pd.concat([WD_2015, WD_2016,WD_2017])
Pr_Total = pd.concat([Pr_2015, Pr_2016,Pr_2017])
R_Total = pd.concat([R_2015, R_2016,R_2017])
Pa_Total = pd.concat([Pa_2015, Pa_2016,Pa_2017])



del O3_2015
del NO_2015
del NO2_2015
del NOX_2015
del SO2_2015
del CO_2015
del PM10_2015
del PM2_5_2015
del TEMP_2015
del RH_2015
del WS_2015
del WD_2015
del Pr_2015
del R_2015
del Pa_2015
del O3_2016
del NO_2016
del NO2_2016
del NOX_2016
del SO2_2016
del CO_2016
del PM10_2016
del PM2_5_2016
del TEMP_2016
del RH_2016
del WS_2016
del WD_2016
del Pr_2016
del R_2016
del Pa_2016
del O3_2017
del NO_2017
del NO2_2017
del NOX_2017
del SO2_2017
del CO_2017
del PM10_2017
del PM2_5_2017
del TEMP_2017
del RH_2017
del WS_2017
del WD_2017
del Pr_2017
del R_2017
del Pa_2017

if True:
    #lista Lim_Sup para límites superiores de contaminantes
    Lim_Medicion = [20, 300, 300, 300, 20 ,11, 75*6, 45*6, 45,100,50,360,1000,1000,1000]
    #Valor límite de la norma 
    Lim_Norma = [0.095, 0.395,0.210,0.395,0.288, 11, 75, 45, 45, 100,30,360, 1000,1000,1000]
    
    O3_Total = VentanaMovil(O3_Total,Lim_Medicion[0],Lim_Norma[0])
    
    NO_Total = VentanaMovil(NO_Total,Lim_Medicion[1],Lim_Norma[1])
    
    NO2_Total = VentanaMovil(NO2_Total,Lim_Medicion[2],Lim_Norma[2])
    
    NOX_Total = VentanaMovil(NOX_Total,Lim_Medicion[3],Lim_Norma[3])
    
    SO2_Total = VentanaMovil(SO2_Total,Lim_Medicion[4],Lim_Norma[4])
    
    CO_Total = VentanaMovil(CO_Total,Lim_Medicion[5],Lim_Norma[5])
    
    PM10_Total = VentanaMovil(PM10_Total,Lim_Medicion[6],Lim_Norma[6])
    
    PM2_5_Total = VentanaMovil(PM2_5_Total,Lim_Medicion[7],Lim_Norma[7])
    
    TEMP_Total = VentanaMovil(TEMP_Total,Lim_Medicion[8],Lim_Norma[8])
    
    RH_Total = VentanaMovil(RH_Total,Lim_Medicion[9],Lim_Norma[9])
    
    WS_Total = VentanaMovil(WS_Total,Lim_Medicion[10],Lim_Norma[10])
    
    WD_Total = VentanaMovil(WD_Total,Lim_Medicion[11],Lim_Norma[11])
    
    Pr_Total = VentanaMovil(Pr_Total,Lim_Medicion[12],Lim_Norma[12])
    
    R_Total = VentanaMovil(R_Total,Lim_Medicion[13],Lim_Norma[13])
    
    Pa_Total = VentanaMovil(Pa_Total,Lim_Medicion[14],Lim_Norma[14])
    

O3_Total = Normalizacion(O3_Total,0)    

NO_Total = Normalizacion(NO_Total,1)

NO2_Total = Normalizacion(NO2_Total,2)

NOX_Total = Normalizacion(NOX_Total,3)

SO2_Total = Normalizacion(SO2_Total,4)

CO_Total = Normalizacion(CO_Total,5)

PM10_Total = Normalizacion(PM10_Total,6)

PM2_5_Total = Normalizacion(PM2_5_Total,7)

TEMP_Total = Normalizacion(TEMP_Total,8)

RH_Total = Normalizacion(RH_Total,9)

WS_Total = Normalizacion(WS_Total,10)

WD_Total =  Normalizacion(WD_Total,11)

Pr_Total =  Normalizacion(Pr_Total,12)

R_Total = Normalizacion(R_Total,13)

Pa_Total = Normalizacion(Pa_Total,14)

O3_Total.to_csv("C:/Users/iq699078/Downloads/Norm/O3_N.csv")
NO_Total.to_csv("C:/Users/iq699078/Downloads/Norm/NO_N.csv")
NO2_Total.to_csv("C:/Users/iq699078/Downloads/Norm/NO2_N.csv")
NOX_Total.to_csv("C:/Users/iq699078/Downloads/Norm/NOX_N.csv")
SO2_Total.to_csv("C:/Users/iq699078/Downloads/Norm/S02_N.csv")
CO_Total.to_csv("C:/Users/iq699078/Downloads/Norm/CO_N.csv")
PM10_Total.to_csv("C:/Users/iq699078/Downloads/Norm/PM10_N.csv")
PM2_5_Total.to_csv("C:/Users/iq699078/Downloads/Norm/PM2_5_N.csv")
TEMP_Total.to_csv("C:/Users/iq699078/Downloads/Norm/TEMP_N.csv")
RH_Total.to_csv("C:/Users/iq699078/Downloads/Norm/HR_N.csv")
WS_Total.to_csv("C:/Users/iq699078/Downloads/Norm/VV_N.csv")
WD_Total.to_csv("C:/Users/iq699078/Downloads/Norm/DV_N.csv")
Pr_Total.to_csv("C:/Users/iq699078/Downloads/Norm/Pr_N.csv")
R_Total.to_csv("C:/Users/iq699078/Downloads/Norm/R_N.csv")
Pa_Total.to_csv("C:/Users/iq699078/Downloads/Norm/Pa_N.csv")



